## Relevent imports

In [1]:
# relevant imports 
import numpy as np
import cv2
import cv2 as cv
import matplotlib.pyplot as plt
import glob

In [ ]:
cv2.VideoWriter()

## Transform

In [2]:
def per_transform(img): # first one
    # Get the image x and y dimensions, will be used in the destination points
    img_size = (img.shape[1],img.shape[0])
    # Identify the source points 
    src = np.float32([[510,460], [750,460],[150,650],[1200,650]])
    #identify the destination points 
    offset = 150
    dst = np.float32([[offset, offset], [img_size[0]-offset, offset], 
                                     [offset, img_size[1]-offset],
                                     [img_size[0]-offset, img_size[1]-offset] 
                                    ])
    # Get the transformation matrix necissary for mapping
    M = cv2.getPerspectiveTransform(src, dst)
    # Get the inverse matrix to transform the image back
    Minv = cv2.getPerspectiveTransform(dst, src)
    # wrap the image
    warped = cv2.warpPerspective(img, M, img_size)
    return warped, M, Minv

In [5]:
def per_transform(img): # recent
    img_size = (img.shape[1],img.shape[0])
    #src = np.float32([[540,424], [500,900],[650,150],[650,1200]])
    src = np.float32([[424,540], [900,500],[150,650],[1200,650]])
    offset = 0
    
    dst = np.float32([[offset, offset], [img_size[0]-offset, offset], 
                                     [offset, img_size[1]-offset],
                                     [img_size[0]-offset, img_size[1]-offset] 
                                    ])
    
    M = cv2.getPerspectiveTransform(src, dst)
    Minv = cv2.getPerspectiveTransform(dst, src)
    warped = cv2.warpPerspective(img, M, img_size)
    return warped, M, Minv

In [4]:
def per_transform(img): # middle
    img_size = (img.shape[1],img.shape[0])
    src = np.float32([[545, 480],[735, 480],
                      [310, 640],[990, 640]])
    dst = np.float32([[310, 350], [1075, 350], 
                     [310, 640],[1075, 640]])
    
    M = cv2.getPerspectiveTransform(src, dst)
    Minv = cv2.getPerspectiveTransform(dst, src)
    warped = cv2.warpPerspective(img, M, img_size)
    return warped, M, Minv

In [6]:
def transform(img,M):
    img_size = (img.shape[1],img.shape[0])
    warped = cv2.warpPerspective(img, M, img_size)
    return warped

## Thresholding techs

In [8]:
def S_thresholder(S, thresh=(0, 255)):

    S_threshold = S * 0
    S_threshold[(S >= thresh[0]) & (S <= thresh[1])] = 1

    return S_threshold


def abs_sobel_thresh(img, orient='x', sobel_kernel=3, thresh=(0, 255)):
    if orient == "x":
        sobel = cv.Sobel(img, cv.CV_64F, 1, 0, ksize=sobel_kernel)
    elif orient == "y":
        sobel = cv.Sobel(img, cv.CV_64F, 0, 1, ksize=sobel_kernel)

    abs_sobel = np.abs(sobel)

    scaled_sobel = np.uint8(255 * abs_sobel / np.max(abs_sobel))

    sbinary = scaled_sobel * 0
    sbinary[(scaled_sobel >= thresh[0]) & (scaled_sobel < thresh[1])] = 1

    return sbinary


# bad one
def binarization_choice1(img):
    S = cv.cvtColor(img, cv.COLOR_BGR2HLS)[:, :, 2]

    sthresh = S_thresholder(S, (90, 255)) * 255
    canny = cv.Canny(cv.GaussianBlur(S, (5, 5), 0), 50, 170)

    binary_sobx_Sthreshs = S * 0
    binary_sobx_Sthreshs[(sthresh == 255) | (canny == 255)] = 255

    return binary_sobx_Sthreshs


def BGR_equlization(frame, B=255, G=255, R=255):
    B_eq = np.uint8(cv.equalizeHist(frame[:, :, 0]) * (B / 255))
    G_eq = np.uint8(cv.equalizeHist(frame[:, :, 1]) * (G / 255))
    R_eq = np.uint8(cv.equalizeHist(frame[:, :, 2]) * (R / 255))

    return cv.merge((B_eq, G_eq, R_eq))


def HLS_equlization(frame, H=255, L=255, S=255):
    frame = cv.cvtColor(frame, cv.COLOR_BGR2HLS)
    H_eq = np.uint8(cv.equalizeHist(frame[:, :, 0]) * (H / 255))
    L_eq = np.uint8(cv.equalizeHist(frame[:, :, 1]) * (L / 255))
    S_eq = np.uint8(cv.equalizeHist(frame[:, :, 2]) * (S / 255))

    return cv.cvtColor(cv.merge((H_eq, L_eq, S_eq)), cv.COLOR_HLS2BGR)


# current
def binarization_choice2(frame):
    frame_equlized_HLS = HLS_equlization(frame, 100, 50, 255)

    S = cv.cvtColor(frame_equlized_HLS, cv.COLOR_BGR2HLS)[:, :, 2]

    sthresh = S_thresholder(S, (140, 230)) * 255

    gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)

    canny = cv.Canny(gray, 40, 80)

    gaussian = cv.GaussianBlur(gray, (9, 9), 0)
    sobelx = abs_sobel_thresh(gaussian, "x", 3, (40, 220)) * 255

    sobelx[canny == 255] = 0
    sobelx = cv.dilate(sobelx, (15, 15))

    binary = S * 0
    binary[(sthresh == 255) | (sobelx == 255)] = 255

    closing = cv.morphologyEx(binary, cv.MORPH_CLOSE, np.ones((7, 7)))

    return closing

In [9]:
def abs_sobel_thresh1(img, orient='x', thresh_min=0, thresh_max=255,fullimage = True):
    # Apply the following steps to img
    # 1) Convert to grayscale
    if fullimage:
        grey = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    else:
        grey = img
    # 2) Take the derivative in x or y given orient = 'x' or 'y'
    if orient == 'x':
        # Here dx = 1 and dy = 0
        gradient = cv2.Sobel(grey,cv2.CV_64F,1,0)
    elif orient == 'y':
        # Here dx = 0 and dy = 1
        gradient = cv2.Sobel(grey,cv2.CV_64F,0,1)
    # For the gradient, the range of output will be from -4*255 to 4*255
    # 3) Take the absolute value of the derivative or gradient, now the range will be from 0 to 4*255
    gradient_abs = abs(gradient)
    # 4) Scale to 8-bit (0 - 255) then convert to type = np.uint8
    scaled_gradient_abs = np.uint8(255*gradient_abs/gradient_abs.max()) #if maximum is 4*255 it will be like dividing by 4
    # 5) Create a mask of 1's where the scaled gradient magnitude 
            # is > thresh_min and < thresh_max
    binary_img = np.zeros_like(scaled_gradient_abs)
    binary_img[(scaled_gradient_abs >= thresh_min) & (scaled_gradient_abs <= thresh_max)] = 1
    # 6) Return this mask as your binary_output image
    #binary_output = np.copy(img) # Remove this line
    return binary_img


In [10]:
def dir_threshold(img, sobel_kernel=3, thresh=(0.85, 1.05)):
    # Grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Calculate the x and y gradients
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    # Take the absolute value of the gradient direction, 
    # apply a threshold, and create a binary image result
    absgraddir = np.arctan2(np.absolute(sobely), np.absolute(sobelx))
    binary_output =  np.zeros_like(absgraddir)
    binary_output[(absgraddir >= thresh[0]) & (absgraddir <= thresh[1])] = 1

    # Return the binary image
    return binary_output

In [11]:
def threshold(image):
    output_sobelx = abs_sobel_thresh1(image, orient='x', thresh_min=20, thresh_max=100)
    output_sobely = abs_sobel_thresh1(image, orient='y', thresh_min=20, thresh_max=100)
    output_dir = dir_threshold(image, sobel_kernel=5, thresh=(0.8, 1.3))
    hsv = cv2.cvtColor(image,cv2.COLOR_BGR2HSV)
    s_hsv = hsv[:,:,1]
    h_hsv = hsv[:,:,0]
    v_hsv = hsv[:,:,2]
    
    combined = np.zeros_like(s_hsv)
    edge_mask = (output_sobely == 1) | (output_sobelx == 1) & (output_dir == 1)
    mask = (s_hsv >= 60)
    mask_unwanted = (v_hsv <= 60)
    combined[edge_mask] = 1
    combined[mask] = 1
    combined[mask_unwanted] = 0
    return combined 

## Lane detection

In [12]:
def find_lane_pixels(binary_warped):
    # Take a histogram of the bottom half of the image
    histogram = np.sum(binary_warped[binary_warped.shape[0]//2:,:], axis=0)
    # Create an output image to draw on and visualize the result
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))
    # Find the peak of the left and right halves of the histogram
    # These will be the starting point for the left and right lines
    midpoint = int(histogram.shape[0]//2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint #argmax returns the index of the max number (vertical sum) of the bottom half image 

    # HYPERPARAMETERS
    # Choose the number of sliding windows
    nwindows = 9
    # Set the width of the windows +/- margin
    margin = 50
    # Set minimum number of pixels found to recenter window
    minpix = 50

    # Set height of windows - based on nwindows above and image shape
    window_height = int(binary_warped.shape[0]//nwindows)
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = binary_warped.nonzero() # return the indicies of the elements that are non zero, on the x and on the y
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    # Current positions to be updated later for each window in nwindows
    leftx_current = leftx_base
    rightx_current = rightx_base

    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []

    # Step through the windows one by one
    for window in range(nwindows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low = binary_warped.shape[0] - (window+1)*window_height
        win_y_high = binary_warped.shape[0] - window*window_height
        win_xleft_low = leftx_current - margin
        win_xleft_high = leftx_current + margin
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin
        
        # Draw the windows on the visualization image
        cv2.rectangle(out_img,(win_xleft_low,win_y_low),
        (win_xleft_high,win_y_high),(0,255,0), 2) 
        cv2.rectangle(out_img,(win_xright_low,win_y_low),
        (win_xright_high,win_y_high),(0,255,0), 2) 
        
        # Identify the nonzero pixels in x and y within the window #
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xleft_low) &  (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xright_low) &  (nonzerox < win_xright_high)).nonzero()[0]
        # good left indicies are the non zero indicies inside the left window
        
        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        
        # If you found > minpix pixels, recenter next window on their mean position
        if len(good_left_inds) > minpix:
            leftx_current = int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:        
            rightx_current = int(np.mean(nonzerox[good_right_inds]))

    # Concatenate the arrays of indices (previously was a list of lists of pixels)
    try:
        left_lane_inds = np.concatenate(left_lane_inds) # left lane indicies are the good left indicies for all windows
        right_lane_inds = np.concatenate(right_lane_inds)
    except ValueError:
        # Avoids an error if the above is not implemented fully
        pass
    #print(left_lane_inds)
    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]
    return leftx, lefty, rightx, righty, out_img


def fit_polynomial(binary_warped):
    # Find our lane pixels first
    leftx, lefty, rightx, righty, out_img = find_lane_pixels(binary_warped)

    # Fit a second order polynomial to each using `np.polyfit`
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)

    # Generate x and y values for plotting
    ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )
    #print(binary_warped.shape)
    #print(ploty)
    try:
        left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
        right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    except TypeError:
        # Avoids an error if `left` and `right_fit` are still none or incorrect
        print('The function failed to fit a line!')
        left_fitx = 1*ploty**2 + 1*ploty
        right_fitx = 1*ploty**2 + 1*ploty

    ## Visualization ##
    # Colors in the left and right lane regions
    out_img[lefty, leftx] = [255, 0, 0]
    out_img[righty, rightx] = [0, 0, 255]

    # Plots the left and right polynomials on the lane lines
    #plt.plot(left_fitx, ploty, color='yellow')
    #plt.plot(right_fitx, ploty, color='yellow')
    for i in range(1,left_fitx.shape[0]):
        cv.line(out_img,(int(left_fitx[i-1]),int(ploty[i-1])),(int(left_fitx[i]),int(ploty[i])),(0,255,255),thickness = 3)
        cv.line(out_img,(int(right_fitx[i-1]),int(ploty[i-1])),(int(right_fitx[i]),int(ploty[i])),(0,255,255),thickness = 3)
    return out_img,left_fit,right_fit

In [16]:
def fit_poly(img_shape, leftx, lefty, rightx, righty,left_fit,right_fit):
     ### TO-DO: Fit a second order polynomial to each with np.polyfit() ###
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)
    # Generate x and y values for plotting
    ploty = np.linspace(0, img_shape[0]-1, img_shape[0])
    ### TO-DO: Calc both polynomials using ploty, left_fit and right_fit ###
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    
    return left_fitx, right_fitx, ploty

def search_around_poly(binary_warped,left_fit,right_fit):
    # HYPERPARAMETER
    # Choose the width of the margin around the previous polynomial to search
    # The quiz grader expects 100 here, but feel free to tune on your own!
    margin = 50

    # Grab activated pixels both on x axis and on y 
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    
    ### TO-DO: Set the area of search based on activated x-values ###
    ### within the +/- margin of our polynomial function ###
    ### Hint: consider the window areas for the similarly named variables ###
    ### in the previous quiz, but change the windows to our new search area ###
    left_lane_inds = ((nonzerox > (left_fit[0]*(nonzeroy**2) + left_fit[1]*nonzeroy + 
                    left_fit[2] - margin)) & (nonzerox < (left_fit[0]*(nonzeroy**2) + 
                    left_fit[1]*nonzeroy + left_fit[2] + margin)))
    right_lane_inds = ((nonzerox > (right_fit[0]*(nonzeroy**2) + right_fit[1]*nonzeroy + 
                    right_fit[2] - margin)) & (nonzerox < (right_fit[0]*(nonzeroy**2) + 
                    right_fit[1]*nonzeroy + right_fit[2] + margin)))
    # good left indicies are the non zero indicies inside the defined area
    
    # Again, extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]

    # Fit new polynomials
    left_fitx, right_fitx, ploty = fit_poly(binary_warped.shape, leftx, lefty, rightx, righty,left_fit,right_fit)
    
    ## Visualization ##
    # Create an image to draw on and an image to show the selection window
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))*255
    window_img = np.zeros_like(out_img)
    # Color in left and right line pixels
    out_img[nonzeroy[left_lane_inds], nonzerox[left_lane_inds]] = [255, 0, 0]
    out_img[nonzeroy[right_lane_inds], nonzerox[right_lane_inds]] = [0, 0, 255]

    # Generate a polygon to illustrate the search window area
    # And recast the x and y points into usable format for cv2.fillPoly()
    left_line_window1 = np.array([np.transpose(np.vstack([left_fitx-margin, ploty]))])
    left_line_window2 = np.array([np.flipud(np.transpose(np.vstack([left_fitx+margin, 
                              ploty])))])
    left_line_pts = np.hstack((left_line_window1, left_line_window2))
    right_line_window1 = np.array([np.transpose(np.vstack([right_fitx-margin, ploty]))])
    right_line_window2 = np.array([np.flipud(np.transpose(np.vstack([right_fitx+margin, 
                              ploty])))])
    right_line_pts = np.hstack((right_line_window1, right_line_window2))

    # Draw the lane onto the warped blank image
    cv2.fillPoly(window_img, np.int_([left_line_pts]), (0,255, 0))
    cv2.fillPoly(window_img, np.int_([right_line_pts]), (0,255, 0))
    result = cv2.addWeighted(out_img, 1, window_img, 0.3, 0)
    
    # Plot the polynomial lines onto the image
    #plt.plot(left_fitx, ploty, color='yellow')
    #plt.plot(right_fitx, ploty, color='yellow')
    ## End visualization steps ##
    for i in range(1,left_fitx.shape[0]):
        cv.line(result,(int(left_fitx[i-1]),int(ploty[i-1])),(int(left_fitx[i]),int(ploty[i])),(0,255,255),thickness = 3)
        cv.line(result,(int(right_fitx[i-1]),int(ploty[i-1])),(int(right_fitx[i]),int(ploty[i])),(0,255,255),thickness = 3)
    
    return result,left_fit,right_fit,ploty

# Run image through the pipeline
# Note that in your project, you'll also want to feed in the previous fits
#result = search_around_poly(binary_warped)

## Drawing relevant info on video

In [13]:
def draw_rectangle(image,left_eqn,right_eqn):
    line_image = np.copy(image)*0 # creating a blank to draw lines on
    #ploty = np.linspace(0, image.shape[0]-1, image.shape[0])
    XX, YY = np.meshgrid(np.arange(0, image.shape[1]), np.arange(0, image.shape[0]))
    region_thresholds = (XX < (right_eqn[0]*YY**2 + right_eqn[1]*YY + right_eqn[2])) & \
                        (XX > (left_eqn[0]*YY**2 + left_eqn[1]*YY + left_eqn[2])) #& \
                        #(YY < (right_eqn[0]*YY**2 + right_eqn[1]*YY + right_eqn[2])) & \
                        #(YY > (left_eqn[0]*YY**2 + left_eqn[1]*YY + left_eqn[2])) 

    line_image[region_thresholds] = (0xb9,0xff,0x99) #dcffcc
    return line_image

In [14]:
def measure_curvature_pixels(left_fit, right_fit):
    ym_per_pix = 30/720 # meters per pixel in y dimension
    xm_per_pix = 3.7/700 # meters per pixel in x dimension
    
    y_eval = 720   # bottom of image
    
    # Calculation of R_curve (radius of curvature)
    left_curverad  = ((1 + (2*(left_fit[0]/xm_per_pix)*y_eval*ym_per_pix + left_fit[1])**2)**1.5) / np.absolute(2*(left_fit[0]/xm_per_pix))
    right_curverad = ((1 + (2*(right_fit[0]/xm_per_pix)*y_eval*ym_per_pix + right_fit[1])**2)**1.5) / np.absolute(2*(right_fit[0]/xm_per_pix))
    
    
    ave_curvature = (left_curverad + right_curverad) / 2
    
    return ave_curvature

In [31]:
# apply the function on videos 
capture = cv2.VideoCapture('project_video.mp4') #capture is instance of the videocapture class that contains the video given

isTrue,frame = capture.read()
output = binarization_choice2(frame)
#output = threshold(frame)
warped, m, minv = per_transform(output)
first_time,left_eqn,right_eqn = fit_polynomial(warped)
rectangle = draw_rectangle(frame,left_eqn,right_eqn)
while True:
    isTrue,frame = capture.read()
    output = binarization_choice2(frame)
    #output = threshold(frame)
    warped = transform(output,m)
    
    output,left_eqn,right_eqn,ploty = search_around_poly(warped,left_eqn,right_eqn)
    curve = measure_curvature_pixels(left_eqn,right_eqn)
    #first_time,left_eqn,right_eqn = fit_polynomial(warped)
    rectangle = draw_rectangle(frame,left_eqn,right_eqn)
    transformed_back = transform(output,minv)
    #correct_rectangle = transform(rectangle,minv)
    cv.putText(frame,"curvature: {} m".format(curve),(255,255), cv.FONT_ITALIC,1.0,(255,255,255),2)
    cv2.imshow('Video',cv.addWeighted(transformed_back,0.5,frame,1,0))
    #cv2.imshow('Video',output*255)
    if cv2.waitKey(16) & 0xFF == ord('q'): 
        break
capture.release()  
cv2.destroyAllWindows() 

In [ ]:
# apply the function on videos 
capture = cv2.VideoCapture('challenge_video.mp4') #capture is instance of the videocapture class that contains the video given

while True:
    isTrue,frame = capture.read()
    
    #cv2.imshow('Video',output)
    if cv2.waitKey(16) & 0xFF == ord('q'): 
        break
capture.release()  
cv2.destroyAllWindows() 

In [30]:
cv2.destroyAllWindows() 